In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from obspy.core.event import read_events
import obspy
import eqcorrscan
import eqcorrscan.core.match_filter.template as template
from obspy.clients.fdsn.client import Client
from eqcorrscan.core.template_gen import template_gen
from eqcorrscan import Tribe
from datetime import datetime
client = Client('IRIS')

import warnings
warnings.filterwarnings('ignore')

In [18]:
clust_name = 'growclust10_ev519520'

In [4]:
# Read in results from all 4 stations
# Compare detections between, see if any overlap
# Compare the detection list to the original catalog, see how many are new/repeated
# Then look into repicking

### Functions below

In [10]:
def read_detections(path,t1,t2):
    
    time_bins = pd.to_datetime(np.arange(t1,t2,pd.Timedelta(1,'days')))
    
    parties = []
    for t in time_bins:
        pfile = path + 'party_' + t.strftime('%m_%d_%Y') + '.tgz'
        party = eqcorrscan.core.match_filter.party.Party().read(pfile)
        if len(party.families) > 0:
            parties.append(party)
        else:
            print('Detection did not work on ' + t.strftime('%m_%d_%Y'))

    # Combine the results from separate days into one party
    families = []
    num_templates = len(parties[0].families)
    for i in range(num_templates):
        fam_list = [p[i] for p in parties]
        fam = fam_list[0]
        for f in fam_list[1:]:
            fam = fam.append(f)
        families.append(fam)
    party = eqcorrscan.core.match_filter.party.Party(families=families)
    
    return party

def pick_corr(phase,sta,chan,t1,t2):
    """
    Cross correlates two phase picks from the same channel and calculates their differential travel times
    This function pulls in the waveform data directly from Iris
    
    INPUT:
    phase: 'P' or 'S'; this determines the size of the window cut around the pick time
    sta: string, station code 
    chan: string, channel code
    t1: pick time of first pick as UTCDatetime
    t2: pick time of second pick as UTCDatetime
    ot1: origin time of the earthquake the first pick is for in UTCDatetime
    ot2: origin time of the earthquake the second pick is for in UTCDatetime
    
    RETURNS:
    value: maximum cross correlation value of the waveform cut around the two picks
    dtt: differential travel times of the two picks
    """
    
    if phase=='Z':
        t_off=[-0.3,0.5]
        t_off=[-1,1.5]
    if phase=='E':
        t_off = [-1,2.5]
        
    # Pull in waveforms longer than picks
    tr1 = client.get_waveforms('NV',sta,'',chan,t1-5,t1+5).detrend()
    tr2 = client.get_waveforms('NV',sta,'',chan,t2-5,t2+5).detrend()
    
    # Filter
    tr1.filter('bandpass',freqmin=8,freqmax=35)
    tr2.filter('bandpass',freqmin=8,freqmax=35)
    
    # Cut the waveforms around picks
    tr1.trim(starttime=t1+t_off[0],endtime=t1+t_off[1])
    tr2.trim(starttime=t2+t_off[0],endtime=t2+t_off[1])
    
    # Cross correlate waveforms, allowing them to shift relative to each other for the case of faulty picks
    xcor = obspy.signal.cross_correlation.correlate(tr1[0],tr2[0],100)
    shift,value = obspy.signal.cross_correlation.xcorr_max(xcor)
    
    
    return(value)


def event_corr(pair):
    """
    INPUT:
    A tuple of indices of events in the catalog
    
    For the two events corresponding to those indices, finds the common picks between them and calculates
    their cross correlation coefficient and differential travel times
    
    RETURNS:
    A numpy array designed to be easily written to a text file in the format needed for GrowClust
    Array has format ((ev1id,ev2id),(commonpick1,commonpick2,...))
    Where each commonpick array has the format (station,differential travel time,cross correlation coefficient,phase)
    """
    
    path = 'sep2017_mseed/'
    
    i = pair[0]
    j = pair[1]

    sta1 = [p.waveform_id.station_code for p in cat[i].picks]
    sta2 = [p.waveform_id.station_code for p in cat[j].picks]
    pha1 = [p.waveform_id.channel_code[2] for p in cat[i].picks]
    pha2 = [p.waveform_id.channel_code[2] for p in cat[j].picks]
    picks1 = [a+'_'+pha1[k] for k,a in enumerate(sta1)]
    picks2 = [a+'_'+pha2[k] for k,a in enumerate(sta2)]
    common_picks = list(set(picks1).intersection(set(picks2)))

    pick_arr = np.empty((len(common_picks),3),dtype=object)
    for k,p in enumerate(common_picks):
        # Find corresponding pick info within each event
        # Send to pick_corr to get information
        sta,phase = p.split('_')

        sta1_ind = [ ind for ind in range(len(sta1)) if sta1[ind] == sta ]
        pha1_ind = [ ind for ind in range(len(pha1)) if pha1[ind] == phase ]
        pick1_ind = list(set(sta1_ind).intersection(set(pha1_ind)))
        pick1 = cat[i].picks[pick1_ind[0]]

        sta2_ind = [ ind for ind in range(len(sta2)) if sta2[ind] == sta ]
        pha2_ind = [ ind for ind in range(len(pha2)) if pha2[ind] == phase ]
        pick2_ind = list(set(sta2_ind).intersection(set(pha2_ind)))
        pick2 = cat[j].picks[pick2_ind[0]]

        chan = pick1.waveform_id.channel_code
        xcor_val = pick_corr(phase,sta,chan,pick1.time,pick2.time)
        # xcor_val,dtt = pick_corr(phase,sta,chan,pick1.time,pick2.time,ot1,ot2)

        # Write station- and phase-specific line
        
        pick_arr[k][0]=sta
        pick_arr[k][1]=xcor_val
        pick_arr[k][2] = phase
        
    
    return(pick_arr)

In [26]:
t1 = datetime(2017,9,1)
t2 = datetime(2017,10,1)

In [ ]:
path = '/Users/zoekrauss/endeavour_clustering/detections_sep2017_kemo/'
party = read_detections(path,t1,t2)

In [ ]:
for f in kemo_party:
    print(f.template.name)
    print(f.template.event)
    print(f.detections)

## Filter to only keep picks with both channels > 0.7 cc

In [38]:
family = [f for f in party if f.template.name==clust_name][0]
evs = [det.event for det in family.detections]
cat = obspy.core.event.Catalog(events=evs)
cat.append(family.template.event)

temp = family.template.event
template_index = -1

keep = []
for i,ev in enumerate(cat):
    vals = event_corr([template_index,i])
    print(vals)

[['KEMO' 1.0000000000000002 'Z']
 ['KEMO' 1.0000000000000002 'E']]
[['KEMO' 0.589220581663239 'Z']
 ['KEMO' 0.5276567822582933 'E']]
[['KEMO' 0.6846925383765539 'Z']
 ['KEMO' 0.5483571389659104 'E']]
[['KEMO' 0.5912708985882392 'Z']
 ['KEMO' 0.5480310202366674 'E']]
[['KEMO' 0.7302350968488929 'Z']
 ['KEMO' 0.5359127952821228 'E']]
[['KEMO' 0.6317324081240476 'Z']
 ['KEMO' 0.47381464982040894 'E']]
[['KEMO' -0.20882566904547908 'Z']
 ['KEMO' 0.3450162870994042 'E']]
[['KEMO' 0.24710502191302433 'Z']
 ['KEMO' -0.20772428225602632 'E']]
[['KEMO' 0.22966831539266316 'Z']
 ['KEMO' 0.3221827993461413 'E']]
[['KEMO' 1.0000000000000002 'Z']
 ['ENWF' 0.9999999999999998 'Z']
 ['NCHR' 0.9999999999999999 'E']
 ['KEMO' 1.0000000000000002 'E']
 ['ENWF' 0.9999999999999996 'E']
 ['NCHR' 1.0000000000000002 'Z']]
